# RNA-sequencing Data Visualisation

*Science Park Study Group (07/05/19)* 
<span style="font-size:smaller;"><span style="font-size:smaller;">Fred White & Susanne Wilken

In [ ]:
install.packages("ggplot2")
install.packages("pheatmap")
source("https://bioconductor.org/biocLite.R")
biocLite("DESeq2")

## Part 1 Preprocessing

In [ ]:
library(ggplot2)
suppressMessages(library(DESeq2, quietly = T))

In [ ]:
#counts <- read.table("C:/Users/world/RNAvis/counts.txt", header = T)
counts <- read.table("https://raw.githubusercontent.com/Fred-White94/RNAseqDataVis/master/data/counts.txt", sep = "\t", header = T)


In [ ]:
counts[1:5,1:5]

In [ ]:
dim(counts)

In [ ]:
head(counts)

In [ ]:
sampleinfo <- data.frame(cbind(c("Control_1","Control_2","Control_3","Drought_1","Drought_2","Drought_3"),c(rep("Control",3),rep("Drought",3))), stringsAsFactors = F)
colnames(sampleinfo) <- c("SampleName","Condition")

In [ ]:
sampleinfo

## Part 2 DESeq2 and PCA plot

### PCA

In [ ]:
counts4DE <- counts[,7:12]
colnames(counts4DE) <- sampleinfo[,1]

In [ ]:
head(counts4DE)

In [ ]:
PCD <- prcomp(t(counts4DE), center = T)

In [ ]:
plot(PCD$x, pch = 20, col = as.factor(sampleinfo$Condition))

In [ ]:
DES <- DESeqDataSetFromMatrix(counts4DE, sampleinfo, ~ Condition)

In [ ]:
DES

In [ ]:
DES <- DESeq(DES, parallel = T)

In [ ]:
res <- results(DES, contrast = c("Condition","Control","Drought"))

In [ ]:
res

In [ ]:
length(which(!(is.na(res$padj))))

In [ ]:
plotDispEsts(DES)

In [ ]:
plotMA(res, main = "Control and Drought Differences in Gene Expression")

In [ ]:
res <- cbind(counts$Geneid, res)

In [ ]:
resordered <- res[order(res$padj),]
head(resordered)



**_Get rid of all NA adjusted p-values_**

In [ ]:
resordered <- resordered[!is.na(resordered$padj),]
dim(resordered)

In [ ]:
install.packages("pheatmap", repos = "https://cloud.r-project.org/")

In [ ]:
library(pheatmap)

In [ ]:
dim(counts[which(counts$Geneid %in% resordered$'counts$Geneid'),])

In [ ]:
FC <- counts[which(counts$Geneid %in% resordered$'counts$Geneid'),]
dim(FC)

In [ ]:
head(FC)

In [ ]:
resordered


In [ ]:
FCf <- counts[which(counts$Geneid %in% resordered[1:25,]$'counts$Geneid'),]
dim(FCf)

In [ ]:
colnames(FCf)[2:7] <- sampleinfo$SampleName
head(FCf)

In [ ]:
rownames(FCf) <- FCf$Geneid

In [ ]:
ScaledCounts <- t(scale(t(FCf[,2:7]), scale = T))

In [ ]:
pheatmap(ScaledCounts)